In [1]:
import random
import networkx as nx
import numpy as np
import karateclub
from tkinter import _flatten
import community
import MGCF

### 1. Generate $K$ and $M$

In [66]:
K = random.randint(5, 10)
M = random.randint(1, 100)
print(K)

10


### 2. Gnerate Nodes

In [67]:
average_cluster_size = random.randint(20, 40)

center_x_list = []
center_y_list = []
for i in range(K):
    center_x_list.append(round(10*random.random(), 4))
    center_y_list.append(round(10*random.random(), 4))
node_groundtruth = {}
node_location = {}
idx = 0
for i in range(K):
    for j in range(average_cluster_size):
        node_groundtruth[idx] = i
        node_location[idx] = [center_x_list[i] + round(random.gauss(0, 1), 4), center_y_list[i] + round(random.gauss(0, 1), 4)]
        idx = idx + 1
N = len(node_groundtruth)
print('节点总数', N)
G = nx.Graph()
G.add_nodes_from(list(range(N)))

# ground_truth = {}
# for k, v in node_groundtruth.items():
#     if v not in ground_truth:
#         ground_truth[v] = [k]
#     else:
#         ground_truth[v].append(k)

节点总数 300


### 3. k-nearest neighborhood

In [68]:
loc_np = np.array(list(node_location.values()))
x_np = loc_np[:, 0].reshape((1, -1))
y_np = loc_np[:, 1].reshape((1, -1))
distance_np = np.square(x_np - x_np.T)+np.square(y_np - y_np.T)
k_nearest = {}
k = int(average_cluster_size/2)
for i in range(N):
    k_near_pre = list(np.argsort(distance_np[i])[:k+1:])
    k_near_pre.remove(i)
    k_nearest[i] = k_near_pre
for i in range(N):
    for j in k_nearest[i]:
        G.add_edge(i, j, weight = np.exp(-1/2*distance_np[i, j]))
        
G.to_undirected()

### 4. Generate $f_{k, m}$

In [69]:
omega = 1/K*np.ones(K)
beta = 1/M*np.ones(M)
F = omega.reshape((1, -1))*beta.reshape((-1, 1))
F = F*K*M
# F [M, K]
print(average_cluster_size)

30


### 5. Allocate for $X$

In [70]:
X = np.zeros((N, M))
for i in range(M):
    for j in range(K):
        tmp = np.random.dirichlet(np.ones(average_cluster_size), size=1)*F[i, j]
        X[j*average_cluster_size:(j+1)*average_cluster_size:, i] = tmp

### 6. for Algorithm Process & Evaluation

In [71]:
# 1. DANMF

model1 = karateclub.DANMF()
model1.fit(G)
cluster_membership = model1.get_memberships()
clusters = cluster_membership
labels_pred = list(_flatten(list(clusters.values())))
partition = dict(zip(list(range(len(labels_pred))), labels_pred))
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
cn = len(partition_2)
U = np.zeros((cn, N))
for k, v in partition.items():
    U[v, k] = 1
    
print(round(community.modularity(partition, G, weight='weight'), 3))
print(round(np.sqrt(np.sum(np.square(np.dot(np.dot(U, X), beta) / np.sum(np.dot(np.dot(U, X), beta))  - np.ones(cn)/cn))), 3))

0.757
0.092


In [72]:
# 2. NNSED

model1 = karateclub.NNSED(dimensions=K)
model1.fit(G)
cluster_membership = model1.get_memberships()
clusters = cluster_membership
labels_pred = list(_flatten(list(clusters.values())))
partition = dict(zip(list(range(len(labels_pred))), labels_pred))
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
cn = len(partition_2)
U = np.zeros((cn, N))
for k, v in partition.items():
    U[v, k] = 1
    
print(round(community.modularity(partition, G, weight='weight'), 3))
print(round(np.sqrt(np.sum(np.square(np.dot(np.dot(U, X), beta) / np.sum(np.dot(np.dot(U, X), beta))  - np.ones(cn)/cn))), 3))

0.616
0.197


In [73]:
# 3. MNMF

model1 = karateclub.MNMF(clusters=K)
model1.fit(G)
cluster_membership = model1.get_memberships()
clusters = cluster_membership
labels_pred = list(_flatten(list(clusters.values())))

remap = list(set(labels_pred))
for i in range(len(labels_pred)):
    labels_pred[i] = remap.index(labels_pred[i])

partition = dict(zip(list(range(len(labels_pred))), labels_pred))
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
cn = len(partition_2)
U = np.zeros((cn, N))
for k, v in partition.items():
    U[v, k] = 1
    
print(round(community.modularity(partition, G, weight='weight'), 3))
print(round(np.sqrt(np.sum(np.square(np.dot(np.dot(U, X), beta) / np.sum(np.dot(np.dot(U, X), beta))  - np.ones(cn)/cn))), 3))

0.748
0.044


In [80]:
# 4. BigClam

model1 = karateclub.BigClam(dimensions=K)
model1.fit(G)
cluster_membership = model1.get_memberships()
clusters = cluster_membership
labels_pred = list(_flatten(list(clusters.values())))

remap = list(set(labels_pred))
for i in range(len(labels_pred)):
    labels_pred[i] = remap.index(labels_pred[i])

partition = dict(zip(list(range(len(labels_pred))), labels_pred))
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
cn = len(partition_2)
U = np.zeros((cn, N))
for k, v in partition.items():
    U[v, k] = 1
    
print(round(community.modularity(partition, G, weight='weight'), 3))
print(round(np.sqrt(np.sum(np.square(np.dot(np.dot(U, X), beta) / np.sum(np.dot(np.dot(U, X), beta))  - np.ones(cn)/cn))), 3))

0.427
0.265


In [81]:
# 5. SNMF

model1 = karateclub.SymmNMF(dimensions=K)
model1.fit(G)
cluster_membership = model1.get_memberships()
clusters = cluster_membership
labels_pred = list(_flatten(list(clusters.values())))

remap = list(set(labels_pred))
for i in range(len(labels_pred)):
    labels_pred[i] = remap.index(labels_pred[i])

partition = dict(zip(list(range(len(labels_pred))), labels_pred))
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
cn = len(partition_2)
U = np.zeros((cn, N))
for k, v in partition.items():
    U[v, k] = 1
    
print(round(community.modularity(partition, G, weight='weight'), 3))
print(round(np.sqrt(np.sum(np.square(np.dot(np.dot(U, X), beta) / np.sum(np.dot(np.dot(U, X), beta))  - np.ones(cn)/cn))), 3))

0.122
0.071


In [82]:
# 6. GEMSEC

model1 = karateclub.GEMSEC(dimensions=K)
model1.fit(G)
cluster_membership = model1.get_memberships()
clusters = cluster_membership
labels_pred = list(_flatten(list(clusters.values())))

remap = list(set(labels_pred))
for i in range(len(labels_pred)):
    labels_pred[i] = remap.index(labels_pred[i])

partition = dict(zip(list(range(len(labels_pred))), labels_pred))
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
cn = len(partition_2)
U = np.zeros((cn, N))
for k, v in partition.items():
    U[v, k] = 1
    
print(round(community.modularity(partition, G, weight='weight'), 3))
print(round(np.sqrt(np.sum(np.square(np.dot(np.dot(U, X), beta) / np.sum(np.dot(np.dot(U, X), beta))  - np.ones(cn)/cn))), 3))

0.713
0.098


In [83]:
# 7. SCD

model1 = karateclub.SCD()
model1.fit(G)
cluster_membership = model1.get_memberships()
clusters = cluster_membership
labels_pred = list(_flatten(list(clusters.values())))

remap = list(set(labels_pred))
for i in range(len(labels_pred)):
    labels_pred[i] = remap.index(labels_pred[i])

partition = dict(zip(list(range(len(labels_pred))), labels_pred))
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
cn = len(partition_2)
U = np.zeros((cn, N))
for k, v in partition.items():
    U[v, k] = 1
    
print(round(community.modularity(partition, G, weight='weight'), 3))
print(round(np.sqrt(np.sum(np.square(np.dot(np.dot(U, X), beta) / np.sum(np.dot(np.dot(U, X), beta))  - np.ones(cn)/cn))), 3))

0.664
0.075


In [84]:
# 8. LPA

model1 = karateclub.LabelPropagation(seed=10, iterations=100)
model1.fit(G)
cluster_membership = model1.get_memberships()
clusters = cluster_membership
labels_pred = list(_flatten(list(clusters.values())))

remap = list(set(labels_pred))
for i in range(len(labels_pred)):
    labels_pred[i] = remap.index(labels_pred[i])

partition = dict(zip(list(range(len(labels_pred))), labels_pred))
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
cn = len(partition_2)
U = np.zeros((cn, N))
for k, v in partition.items():
    U[v, k] = 1
    
print(round(community.modularity(partition, G, weight='weight'), 3))
print(round(np.sqrt(np.sum(np.square(np.dot(np.dot(U, X), beta) / np.sum(np.dot(np.dot(U, X), beta))  - np.ones(cn)/cn))), 3))

0.015
0.121


In [85]:
# 9. Louvain

partition = community.best_partition(G, weight='weight')
labels_pred = partition
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
cn = len(partition_2)
U = np.zeros((cn, N))
for k, v in partition.items():
    U[v, k] = 1
    
print(round(community.modularity(partition, G, weight='weight'), 3))
print(round(np.sqrt(np.sum(np.square(np.dot(np.dot(U, X), beta) / np.sum(np.dot(np.dot(U, X), beta))  - np.ones(cn)/cn))), 3))

0.771
0.073


In [87]:
# Ours
partition = MGCF.main(clusters_number=K, feature=X, multi_graph=[G], alpha_1=1, alpha_2=0.5, alpha_3=0.01, weight_list=[1], alpha=2, epoch_num=250)

Epoch:  0 ; soft_mod:  0.012083 ; loss_cons:  0.089696 ; loss_clus:  0.0
Epoch:  10 ; soft_mod:  7.2e-05 ; loss_cons:  0.075869 ; loss_clus:  0.950196
Epoch:  20 ; soft_mod:  0.03136 ; loss_cons:  0.046896 ; loss_clus:  1.106303
Epoch:  30 ; soft_mod:  0.071691 ; loss_cons:  0.054064 ; loss_clus:  0.977592
Epoch:  40 ; soft_mod:  0.091462 ; loss_cons:  0.035808 ; loss_clus:  0.929335
Epoch:  50 ; soft_mod:  0.178104 ; loss_cons:  0.053095 ; loss_clus:  0.827193
Epoch:  60 ; soft_mod:  0.245927 ; loss_cons:  0.041789 ; loss_clus:  0.790424
Epoch:  70 ; soft_mod:  0.354728 ; loss_cons:  0.049912 ; loss_clus:  0.842123
Epoch:  80 ; soft_mod:  0.500364 ; loss_cons:  0.042412 ; loss_clus:  0.85967
Epoch:  90 ; soft_mod:  0.59725 ; loss_cons:  0.039496 ; loss_clus:  0.904253
Epoch:  100 ; soft_mod:  0.708925 ; loss_cons:  0.040856 ; loss_clus:  1.069692
Epoch:  110 ; soft_mod:  0.743065 ; loss_cons:  0.042174 ; loss_clus:  1.311101
Epoch:  120 ; soft_mod:  0.764208 ; loss_cons:  0.038855 ; l

In [88]:
labels_pred = partition
partition_2 = {}
for i in range(len(labels_pred)):
    if labels_pred[i] not in partition_2:
        partition_2[labels_pred[i]] = [i]
    else:
        partition_2[labels_pred[i]].append(i)
        
cn = len(partition_2)
U = np.zeros((cn, N))
for k, v in partition.items():
    U[v, k] = 1
    
print(round(community.modularity(partition, G, weight='weight'), 3))
print(round(np.sqrt(np.sum(np.square(np.dot(np.dot(U, X), beta) / np.sum(np.dot(np.dot(U, X), beta))  - np.ones(cn)/cn))), 3))

0.774
0.065
